In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import avar
# %matplotlib notebook
%matplotlib qt
import seaborn as sns
sns.set()
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'

path = os.getcwd() + "\\apple_watch_data"
csv_files = glob.glob(os.path.join(path, "*.dat"))
data = pd.read_csv(csv_files[14], names=['time','x','y','z'], delim_whitespace=True)
# data = pd.read_csv('apple_watch_data/1444079161.m_watch_acc.dat', names=['time','x','y','z'], delim_whitespace=True)
print(data)

        time      x      y       z
0      122.0    8.0  -32.0 -1040.0
1      162.0    8.0  -24.0 -1032.0
2      202.0   40.0  -40.0 -1056.0
3      242.0    8.0  -40.0 -1024.0
4      282.0    0.0  -32.0 -1032.0
..       ...    ...    ...     ...
495  19759.0 -120.0 -232.0  -992.0
496  19799.0 -128.0 -240.0  -912.0
497  19839.0 -128.0 -240.0 -1008.0
498  19879.0 -128.0 -248.0  -952.0
499  19919.0 -128.0 -256.0  -992.0

[500 rows x 4 columns]


# Allan Variance Analysis

In [6]:
fig = plt.figure()
fig.set_size_inches(6, 4)
ax = fig.add_subplot(2, 1, 1)

ax.plot(data['x'])
ax.plot(data['y'])
plt.plot(data['z'])
ax.set_xlabel('Timestep [25 Hz]')
ax.set_ylabel('acceleration [mG]')
ax.legend(['x','y','z'])


ct = avar.CharacteristicScale()

ax = fig.add_subplot(2, 1, 2)
ax.plot(ct.allanvar(data['x']))
ax.plot(ct.allanvar(data['y']))
plt.plot(ct.allanvar(data['z']))
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Timescale [# of samples]')
ax.set_ylabel('$\sigma_A^2$')
ax.legend(['x','y'])
fig.tight_layout()

# Dynamic Allan Variance Analysis

In [4]:
from scipy.ndimage import gaussian_filter1d

fig, axs = plt.subplots(2)
horizons = [50, 100, 200]
horizons = [150]

for h in horizons:
    char = avar.CharacteristicScale(horizon=h)
    DAVAR = []
    M_c = []
    MIN_value = []
    for day, v in enumerate(data['x']):
        char.memory.append(v)
        if day > h:
            davar = np.array(char.dynallanvar())
            davar = gaussian_filter1d(davar, 3)
            m_c, min_value = char.find_charecteristic_scale(davar)
            DAVAR.append(davar)
            M_c.append(m_c)
            MIN_value.append(min_value)
        else:
            DAVAR.append(np.zeros(len(char.taus)))
            MIN_value.append(0)
            M_c.append(1)
    axs[1].plot(np.linspace(0,20,len(data['x'])),M_c, label='horizon = '+str(h))
    axs[1].set_xlabel('Time [s]')
    axs[1].set_ylabel('Timescale [# of samples]')
    axs[1].set_yscale("log")
    axs[1].legend()
axs[0].plot(np.linspace(0,20,len(data['x'])),data['x'])
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('x acceleration [mG]')
fig.tight_layout()
plt.savefig('3d_davar.svg')
plt.show()

500

In [5]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import numpy as np
# plt.rcParams['mathtext.fontset'] = 'stix'
# plt.rcParams['font.family'] = 'STIXGeneral'
# plt.rcParams['mathtext.fontset'] = 'custom'
# plt.rcParams['mathtext.rm'] = 'Bitstream Vera Sans'
# plt.rcParams['mathtext.it'] = 'Bitstream Vera Sans:italic'
# plt.rcParams['mathtext.bf'] = 'Bitstream Vera Sans:bold'

DAVAR = np.array(DAVAR)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

timescales = np.linspace(1,DAVAR.shape[1],DAVAR.shape[1])
times = np.linspace(1,20,DAVAR.shape[0])
timescales_mesh, times_mesh = np.meshgrid(timescales,times)
ax.plot_wireframe(np.log(timescales_mesh), times_mesh, np.log(DAVAR), rstride=2, cstride=100, lw=0.7)
ax.plot(np.log(M_c),times,np.log(MIN_value),color='red')
ax.set_xlabel('Timescale [log(# of samples)]')
ax.set_ylabel('Time [s]')
ax.set_zlabel('log($\sigma_A^2$)')
plt.savefig('3d_davar.svg')
plt.show()

<ipython-input-5-b0a16d9f3b20>:18: RuntimeWarning: divide by zero encountered in log
  ax.plot_wireframe(np.log(timescales_mesh), times_mesh, np.log(DAVAR), rstride=2, cstride=100, lw=0.7)
<ipython-input-5-b0a16d9f3b20>:19: RuntimeWarning: divide by zero encountered in log
  ax.plot(np.log(M_c),times,np.log(MIN_value),color='red')
C:\Users\Hossein_Haeri\Anaconda3\lib\site-packages\mpl_toolkits\mplot3d\proj3d.py:109: RuntimeWarning: invalid value encountered in true_divide
  txs, tys, tzs = vecw[0]/w, vecw[1]/w, vecw[2]/w


In [45]:
# DAVAR = np.array(DAVAR)
# DAVAR.shape
print(len(M_c),len(times),len(MIN_value))

500 500 449


Text(0.5, 0.92, 'ABC123 vs $\\mathrm{ABC123}^{123}$')